# Capstone Project - Automatic Ticket Assignment
## Validation notebook


### Import Required Libraries

##### Installing googletrans. It is not avaialble out of box in GoogleTrans

In [1]:
pip install googletrans


     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 952kB 8.8MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=3aa1210e32828c71c196f1753f93be9ca6a720fd3f02be74b0410da1a31fadd0
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=f2aa181d2428ff41e07a4dfcd0e1b78e89bf6834413be55ae40282ed31db41f6
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


##### Importing NTLK package and downloading the Wordnet corpus

In [2]:
import nltk
nltk.download('wordnet')
  

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

##### Importing rest of the libarries needed to use this notebook

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt

import nltk;
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
import string
from googletrans import Translator
pd.set_option('display.width', -1)

random_seed=22
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from tensorflow.keras import optimizers, backend
from tensorflow.keras import models

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.corpus import wordnet   

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
wn = nltk.WordNetLemmatizer()
import string
import re
import pickle
import os

from textblob import TextBlob 


##### mapping google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##### Setting the path for the resources

In [5]:
project_path=os.getcwd()
project_path='/content/drive/My Drive/Projects/'
print(project_path)
#project_path="F:\\StudyProject\\Projects\\Final_Team_project\\"


/content/drive/My Drive/Projects/


### Defining various functions needed to predict using the Deep Learning Model

In [6]:
### Getting the saved model with architecture and weights
def get_model():
  loaded_model=models.load_model(project_path)
  return loaded_model

### loading the Tokenizer used for training the model
### used to create tokens from the string/text passed into the model
def load_tokenizer():
  # loading tokenizer
  with open(project_path+'/resources/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
  return tokenizer

### Routine to translate the input
def translate(text):
  translator = Translator()
  val = translator.translate(text)
  return val

### Routine to apply the Regex to the inputs
def applyRegEx(dd):
    symbol = """!#$%^`&*();:\t\\\"!\{\}\[\]<>-\?\-\\\"—\.,1234567890"""
    ### regex function to clean up symbols and some patterns like emails, disclaimer, numbers, hostnames, urls, etc
    dout=re.sub(' +', ' ' ,
                          re.sub('[^a-zA-z0-9\s]',' ',
                          re.sub("\_ ",' ',
                          re.sub("\\\\",' ',
                          re.sub('\\[a-zA-Z0-9]+',' ',
                          re.sub('(\__)+',' ',
                          re.sub('[{}]'.format(symbol),' ',  
                          re.sub('ü','u',
                          re.sub('ã','a',
                          re.sub('€','e',
                          re.sub('Â','a',
                          re.sub("\'ll", ' will',
                          re.sub("\'ve", ' have',
                          re.sub("\'s", ' is',
                          re.sub("doesn\'t",' does not', 
                          re.sub("cant\'t'",' cannot',       
                          re.sub("cant", ' cannot',
                          re.sub('[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',' ',
                          re.sub('[a-zA-Z0-9_.+-]+[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',' ', 
                          re.sub('/[a-zA-Z0-9_.+-]+',' ',
                          re.sub('\_[0-9]+',' ',       
                          re.sub("\(yes\/no\/na\)",' ',
                          re.sub("received from:",' ',
                          re.sub('\_\_([a-zA-Z0-9])\_\_',' ',
                          re.sub('diese mitteilung ist einzig und allein für die nutzung durch den adressaten bestimmt und kann informationen enthalten, die schutzwürdig, vertraulich oder nach geltendem recht von der offenlegung ausgenommen sind. die verbreitung, verteilung oder vervielfältigung dieser mitteilung durch personen, bei denen es sich nicht um die beabsichtigten empfänger handelt, ist streng verboten. wenn diese mitteilung aufgrund eines versehens bei ihnen eingegangen ist, dann benachrichtigen sie bitte den absender und löschen sie diese mitteilung.',' ',
                          re.sub('本通信は記載された人物のみを対象として使用されるものとし、特権的かつ機密情報を含み、適用される法律に基づいた情報開示から除外されるものとします。正当な受取人以外のいかなる第三者による本通信の配布、流通、再販は固く禁じられています。また本通信が誤って送信された場合は、送信者に連絡の上、本通信を削除してください。',' ',
                          re.sub('this communication is intended solely for the use of the addressee and may contain information that is worthy of protection, confidential or excluded from disclosure under applicable law. the distribution, distribution or reproduction of this communication by persons other than the intended recipients is strictly prohibited. if you have received this message by mistake, please notify the sender and delete this message.',' ',
                          re.sub('neste mensaje está destinado al uso exclusivo de la persona a quien está dirigido y puede contener información privilegiada, confidencial y que está exenta de ser revelada conforme con lo dispuesto en la legislación vigente. toda difusión, distribución o reproducción de este mensaje por parte de otra persona que no sea el receptor al que está destinado queda estrictamente prohibida. si recibe este mensaje por error, se ruega que lo notifique al remitente y borre este mensaje.',' ',
                          re.sub('select the following link to view the disclaimer in an alternate language.',' ',       
                          re.sub('company\/posts',' ',       
                          re.sub('[}/:\r\n]',' ',dd)))))))))))))))))))))))))))))))
    dout=dout.lower()
    return dout


#### Routine to load the custom_stopwords used to clean the data
def load_stopwords():
  pickle_off = open (project_path+"resources/custom_stopwords.txt", "rb")
  custom_stopwords = pickle.load(pickle_off)
  return custom_stopwords

def load_callerlist():
  pickle_off = open (project_path+"resources/callerlist.txt", "rb")
  callerlist = pickle.load(pickle_off)
  return callerlist
  
#### main pipeline to pre-process the input
def cleantext(intext,debug):    
  if debug=='Y':
    print('original ',intext)
  text=applyRegEx(intext)
  if debug=='Y':
    print('regex ',intext)
  text = [word for word in re.split(' ',text) if word not in callerlist]
  if debug=='Y':
    print('caller ',text)
  #text =' '.join(text)
  #text = [replacementset[word] if word in replacementset else word for word in text.split(" ")]
  if debug=='Y':
    print('replacement ',text)
  if debug=='Y':
    print('delete ',text)
  text =' '.join(text)
  text = [word for word in text.split(" ") if word not in custom_stopwords]
  if debug=='Y':
    print('stop ',text)
  text = [word for word in text if not word=='']
  if debug=='Y':
    print('empty ',text)
  text = [wn.lemmatize(word) for word in text]
  if debug=='Y':
    print('lemat ',text)
  text = " ".join([word for word in text if not len(word)<3])
  if debug=='Y':
    print('small ',text)
  return text

### Converting to sequences to input to the model
MAXLEN=300
def get_seq(in_text):
  sequences = tokenizer.texts_to_sequences([in_text])
  ptext = pad_sequences(sequences, maxlen=MAXLEN,padding='post')
  return ptext

### Loading various objects needed for the model

In [7]:
custom_stopwords=load_stopwords()
callerlist=load_callerlist()
tokenizer=load_tokenizer()
loaded_model=models.load_model(project_path)
loaded_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          3000300   
_________________________________________________________________
dropout (Dropout)            (None, 300, 300)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 300, 64)           96064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 75, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 75, 300)           438000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 300)           0         
_________________________________________________________________
flatten (Flatten)            (None, 22500)             0

In [8]:
txt="hr_tool not opening"
cleantext(txt,'Y')

original  hr_tool not opening
regex  hr_tool not opening
caller  ['hr_tool', 'not', 'opening']
replacement  ['hr_tool', 'not', 'opening']
delete  ['hr_tool', 'not', 'opening']
stop  ['hr_tool', 'opening']
empty  ['hr_tool', 'opening']
lemat  ['hr_tool', 'opening']
small  hr_tool opening


'hr_tool opening'

### Evaluate the Model using the "evaluate_text"

In [9]:
y_pred=[]
evaluate_text ="monitoring tool hostname"
print("Input data : ",evaluate_text)
ctext=cleantext(evaluate_text,'N')
print(ctext)
print(get_seq(ctext))
print(len(ctext))
y_pred=loaded_model.predict(get_seq(ctext))
print('--------------------------')
print("Predicted Class : ", np.argmax(y_pred, axis=1)[0])
print('--------------------------')

Input data :  monitoring tool hostname
monitoring tool hostname
[[182   2  11   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   

END of file